In [1]:
# analytics
import pandas as pd 
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
#spatial 
import osmnx as ox
import geopandas as gpd
import contextily as cx
# plotting 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
#settings
import warnings

# set dataframe outputs to three digits 
pd.set_option("display.precision", 3)
#suppress warnings
warnings.filterwarnings('ignore')

### Estimation models


M1: adaptation ~ perception + X * beta

M2: adaptation ~ awareness + X * beta

M3: adaptation ~ experience + X * beta

M4: adaptation ~ floodzone + X * beta 


beta = vector (age, homeownership, efficacy, insurance)

In [2]:
#import processed data
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_k.csv'
df = pd.read_csv(path)
df.shape

(385, 36)

In [3]:
# add hurricane dummy 
hurricane_states = ['Florida','Texas','Louisiana','South Carolina','Virginia','Delaware','New York', 'New Jersey']
df['hurricane'] = [1 if s in hurricane_states else 0 for s in df.state]

In [4]:
def r_square(model):
    # McKelvay-Zavoina
    xb = model.predict(linear=True) #fitted latent value
    var_xb = np.var(xb,ddof=1) # variance of xb
    r2_mz = var_xb / (var_xb + 1) # McKelvay-Zavoina R_2
    # McFadden
    r2_mf = model.prsquared
    return r2_mz

In [5]:
def probit(functions, data):
    results_list = []
    for func in functions:
        model = smf.probit(formula=func, data=data).fit(disp=0)
        df_model = pd.DataFrame({
            'effect': model.params,               
            'p': model.pvalues,    
            'pseudoR_2': r_square(model),
            'LLPr': model.llr_pvalue,
            'BIC': model.bic  
        })
        df_marginal = model.get_margeff().summary_frame()
        df_model = pd.concat([df_model, df_marginal], axis =1)

        df_model.index = pd.MultiIndex.from_product([[func], df_model.index], names=['function', 'beta'])
        results_list.append(df_model)
    results = pd.concat(results_list)
    return results

In [6]:
#duplicate but with logit
def logit(functions, data):
    results_list = []
    for func in functions:
        model = smf.logit(formula=func, data=data).fit(disp=0)
        marg_effects = model.get_margeff().summary_frame()

        df_model = pd.DataFrame({
            'effect': model.params,               
            'p': model.pvalues,                   
            'marginal_effect': marg_effects['dy/dx'],
            'pseudoR_2': model.prsquared,
            'LLPr': model.llr_pvalue,
            'BIC': model.bic  
        })
        df_model.index = pd.MultiIndex.from_product([[func], df_model.index], names=['function', 'beta'])
        results_list.append(df_model)
    results = pd.concat(results_list)
    return results

In [7]:
# functions 
M1 = [
    'made_safer ~ hurricane + perception + age + homeownership + efficacy + insured',
    'documents ~ hurricane + perception + age + homeownership + efficacy + insured',
    'learned_routes ~ hurricane + perception + age + homeownership + efficacy + insured',
    'supplies ~ hurricane + perception + age + homeownership + efficacy + insured',
    'involved ~ hurricane + perception + age + homeownership + efficacy + insured',
    'made_plan ~ hurricane + perception + age + homeownership + efficacy + insured',
    'practiced_drills ~ hurricane + perception + age + homeownership + efficacy + insured',
    'alerts ~ hurricane + perception + age + homeownership + efficacy + insured',
    'family_communication ~ hurricane + perception + age + homeownership + efficacy + insured',
]
M2 = [
    'made_safer ~ hurricane + awareness + age + homeownership + efficacy + insured',
    'documents ~ hurricane + awareness + age + homeownership + efficacy + insured',
    'learned_routes ~ hurricane + awareness + age + homeownership + efficacy + insured',
    'supplies ~ hurricane + awareness + age + homeownership + efficacy + insured',
    'involved ~ hurricane + awareness + age + homeownership + efficacy + insured',
    'made_plan ~ hurricane + awareness + age + homeownership + efficacy + insured',
    'practiced_drills ~ hurricane + awareness + age + homeownership + efficacy + insured',
    'alerts ~ hurricane + awareness + age + homeownership + efficacy + insured',
    'family_communication ~ hurricane + awareness + age + homeownership + efficacy + insured',
]

M3 = [
    'made_safer ~ hurricane + experience + age + homeownership + efficacy + insured',
    'documents ~ hurricane + experience + age + homeownership + efficacy + insured',
    'learned_routes ~ hurricane + experience + age + homeownership + efficacy + insured',
    'supplies ~ hurricane + experience + age + homeownership + efficacy + insured',
    'involved ~ hurricane + experience + age + homeownership + efficacy + insured',
    'made_plan ~ hurricane + experience + age + homeownership + efficacy + insured',
    'practiced_drills ~ hurricane + experience + age + homeownership + efficacy + insured',
    'alerts ~ hurricane + experience + age + homeownership + efficacy + insured',
    'family_communication ~ hurricane + experience + age + homeownership + efficacy + insured',
]

M4 = [
    'made_safer ~ hurricane + floodzone + age + homeownership + efficacy + insured',
    'documents ~ hurricane + floodzone + age + homeownership + efficacy + insured',
    'learned_routes ~ hurricane + floodzone + age + homeownership + efficacy + insured',
    'supplies ~ hurricane + floodzone + age + homeownership + efficacy + insured',
    'involved ~ hurricane + floodzone + age + homeownership + efficacy + insured',
    'made_plan ~ hurricane + floodzone + age + homeownership + efficacy + insured',
    'practiced_drills ~ hurricane + floodzone + age + homeownership + efficacy + insured',
    'alerts ~ hurricane + floodzone + age + homeownership + efficacy + insured',
    'family_communication ~ hurricane + floodzone + age + homeownership + efficacy + insured',
]

In [8]:
# probit regression
M1 = probit(functions = M1, data = df)
M2 = probit(functions = M2, data = df)
M3 = probit(functions = M3, data = df)
M4 = probit(functions = M4, data = df)

In [9]:
# store results 
M1, M2, M3, M4 = M1.round(3), M2.round(3), M3.round(3), M4.round(3)
M1.to_excel('results/M1_probit.xlsx')
M2.to_excel('results/M2_probit.xlsx')
M3.to_excel('results/M3_probit.xlsx')
M4.to_excel('results/M4_probit.xlsx')

### Let's repeat without dummy

In [14]:
# functions 
M1 = [
    'made_safer ~ perception + age + homeownership + efficacy + insured',
    'documents ~ perception + age + homeownership + efficacy + insured',
    'learned_routes ~ perception + age + homeownership + efficacy + insured',
    'supplies ~ perception + age + homeownership + efficacy + insured',
    'involved ~ perception + age + homeownership + efficacy + insured',
    'made_plan ~ perception + age + homeownership + efficacy + insured',
    'practiced_drills ~ perception + age + homeownership + efficacy + insured',
    'alerts ~ perception + age + homeownership + efficacy + insured',
    'family_communication ~ perception + age + homeownership + efficacy + insured',
]
M2 = [
    'made_safer ~  awareness + age + homeownership + efficacy + insured',
    'documents ~ awareness + age + homeownership + efficacy + insured',
    'learned_routes ~ awareness + age + homeownership + efficacy + insured',
    'supplies ~ awareness + age + homeownership + efficacy + insured',
    'involved ~ awareness + age + homeownership + efficacy + insured',
    'made_plan ~ awareness + age + homeownership + efficacy + insured',
    'practiced_drills ~ awareness + age + homeownership + efficacy + insured',
    'alerts ~ awareness + age + homeownership + efficacy + insured',
    'family_communication ~ awareness + age + homeownership + efficacy + insured',
]

M3 = [
    'made_safer ~ experience + age + homeownership + efficacy + insured',
    'documents ~ experience + age + homeownership + efficacy + insured',
    'learned_routes ~ experience + age + homeownership + efficacy + insured',
    'supplies ~ experience + age + homeownership + efficacy + insured',
    'involved ~ experience + age + homeownership + efficacy + insured',
    'made_plan ~ experience + age + homeownership + efficacy + insured',
    'practiced_drills ~ experience + age + homeownership + efficacy + insured',
    'alerts ~ experience + age + homeownership + efficacy + insured',
    'family_communication ~ experience + age + homeownership + efficacy + insured',
]

M4 = [
    'made_safer ~ floodzone + age + homeownership + efficacy + insured',
    'documents ~ floodzone + age + homeownership + efficacy + insured',
    'learned_routes ~ floodzone + age + homeownership + efficacy + insured',
    'supplies ~ floodzone + age + homeownership + efficacy + insured',
    'involved ~ floodzone + age + homeownership + efficacy + insured',
    'made_plan ~ floodzone + age + homeownership + efficacy + insured',
    'practiced_drills ~ floodzone + age + homeownership + efficacy + insured',
    'alerts ~ floodzone + age + homeownership + efficacy + insured',
    'family_communication ~ floodzone + age + homeownership + efficacy + insured',
]

In [15]:
# probit regression
M1 = probit(functions = M1, data = df)
M2 = probit(functions = M2, data = df)
M3 = probit(functions = M3, data = df)
M4 = probit(functions = M4, data = df)

In [16]:
# store results 
M1, M2, M3, M4 = M1.round(3), M2.round(3), M3.round(3), M4.round(3)
M1.to_excel('results/M1_sensitivity.xlsx')
M2.to_excel('results/M2_sensitivity.xlsx')
M3.to_excel('results/M3_sensitivity.xlsx')
M4.to_excel('results/M4_sensitivity.xlsx')